In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

from KAGGLE_NLP_with_disaster_tweets.model import base_model, utils, constants
from KAGGLE_NLP_with_disaster_tweets.data_preparation.utils import DataPipeline, BatchPipeline

from tensorboard import notebook

In [2]:
train_file_name = 'train.csv'
test_file_name = 'test.csv'
sample_submission_file_name = 'sample_submission.csv'

In [3]:
data_pipeline = DataPipeline(train_file_name, test_file_name, sample_submission_file_name)
dataset, submission_test_dataset = data_pipeline.prepare_datasets()

2022-01-26 20:36:19.027432: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-26 20:36:19.027524: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: sersery-asusVivo
2022-01-26 20:36:19.027545: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: sersery-asusVivo
2022-01-26 20:36:19.027763: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 495.29.5
2022-01-26 20:36:19.027829: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.29.5
2022-01-26 20:36:19.027847: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 495.29.5
2022-01-26 20:36:19.028348: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU 

Getting the file: ../data/train.csv
-----------------------------------------------------------------------------------------
Dataset 
Size: 7613
Dataset examples:
Input: [ 582 1716    1    1    1  662   99  145   87    1  662    1    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
Target: [0. 1.]
Input: [ 209  813  539    8 3344    2  209    5   13 2051    1  743    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    

In [4]:
balanced_class_weights = utils.get_balanced_class_weights(data_pipeline.dataframe)

In [5]:
log_directory = "../logs/hyperparameter_tuning/"
utils.start_logging(log_directory, constants.hyperparameters, constants.metrics)

In [6]:
# look for the best hyperparameters
for optimizer in constants.hyperparameters["optimizer"].domain.values:
    for hidden_unit in constants.hyperparameters["hidden_unit"].domain.values:
        for batch_size in constants.hyperparameters["batch_size"].domain.values:
            for learning_rate in (constants.hyperparameters["learning_rate"].domain.min_value,
                                  constants.hyperparameters["learning_rate"].domain.max_value):
                for dropout in constants.hyperparameters["dropout"].domain.values:
                    for class_weights in constants.hyperparameters["class_weights"].domain.values:
                        hparams = {
                            constants.hyperparameters["optimizer"]: optimizer,
                            constants.hyperparameters["hidden_unit"]: hidden_unit,
                            constants.hyperparameters["batch_size"]: batch_size,
                            constants.hyperparameters["learning_rate"]: learning_rate,
                            constants.hyperparameters["class_weights"]: class_weights,
                            constants.hyperparameters["dropout"]: dropout,
                        }
                        model = base_model.BaseModel(
                           batch_pipeline=BatchPipeline(dataset, submission_test_dataset, batch_size),
                           parameters=constants.parameters,
                           hyperparameters=constants.hyperparameters,
                           hparams=hparams,
                           class_weights=balanced_class_weights)

                        with tf.summary.create_file_writer(f'{log_directory}{model.run_name}').as_default():
                            hp.hparams(hparams)
                            accuracy, precision, recall, f1 = model.fit_and_evaluate(log_directory=log_directory)
                            tf.summary.scalar("accuracy", accuracy, step=1)
                            tf.summary.scalar("precision", precision, step=1)
                            tf.summary.scalar("recall", recall, step=1)
                            tf.summary.scalar("f1", f1, step=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320064    
                                                                 
 global_average_pooling1d (G  (None, 64)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
 dense_1 (Dense)             (None, 2)                 18        
                                                                 
Total params: 320,602
Trainable params: 320,602
Non-trainable params: 0
__________________________________________________

2022-01-26 20:36:29.807727: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 76815360 exceeds 10% of free system memory.


380/380 [==============================] - 1s 4ms/step - loss: 0.3884 - accuracy: 0.8424 - val_loss: 0.3368 - val_accuracy: 0.8697
Epoch 3/3
 32/380 [=>............................] - ETA: 1s - loss: 0.3317 - accuracy: 0.8613

2022-01-26 20:36:31.216436: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 76815360 exceeds 10% of free system memory.


380/380 [==============================] - 1s 4ms/step - loss: 0.3091 - accuracy: 0.8762 - val_loss: 0.2899 - val_accuracy: 0.8923


...

2022-01-26 20:36:38.569534: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 76815360 exceeds 10% of free system memory.


380/380 [==============================] - 1s 4ms/step - loss: 0.3870 - accuracy: 0.8385 - val_loss: 0.3317 - val_accuracy: 0.8710
Epoch 3/3
 32/380 [=>............................] - ETA: 1s - loss: 0.3567 - accuracy: 0.8359

2022-01-26 20:36:40.016552: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 76815360 exceeds 10% of free system memory.


47/47 [==============================] - 1s 1ms/step - loss: 0.2901 - accuracy: 0.9003
run=0__lr=0.001__hidden_unit=8__batch_size=16__optimizer=adam__class_weights=none__dropout=0.1 completed.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          320064    
                                                                 
 global_average_pooling1d_2   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 8)                 520       
                                                                 
 dense_5 (Dense)             (None, 2)                 18  

KeyboardInterrupt: 

In [ ]:
%load_ext tensorboard

In [ ]:
tensorboard --logdir logs/hyperparameter_tuning --port 5000